In [38]:
from pathlib import Path
import json
import pandas as pd
from uau_api.settings import Settings
from IPython.display import display, HTML
from requests.compat import urlparse, unquote
import re

In [3]:
wd = Path(Settings().WORKDIR)

In [8]:
with open(wd / 'NotasUAU/downloaded.jsonl') as fc:
    file_contents = [json.loads(line) for line in fc.readlines()]

In [9]:
df = pd.json_normalize(file_contents)

In [45]:
def get_file_id(string):
    query = urlparse(string).query
    result = unquote(query) #273-420A-2-2024-04-10-00289348000736-63023-0.jpg
    return result.split('&', 1)[0].rsplit('/')[-1].split('.')[0]

In [46]:
html = df.iloc[:10][["Empresa", "Obra", "to_download.Processo", "url"]].to_html(
    formatters={'url': lambda x: f'<a href="{x}">{get_file_id(x)}</a>'},
    escape=False
)
display(HTML(html))

,Empresa,Obra,to_download.Processo,url
0,273,420A,2,273-420A-2-2024-04-10-00289348000736-63023-0
1,273,420A,2,273-420A-2-2024-04-10-00289348000736-63023-1
2,273,420A,2,273-420A-2-2024-04-10-00289348000736-63023-2
3,273,420A,3,273-420A-3-2024-04-10-65793192000186-175108-0
4,273,420A,3,273-420A-3-2024-04-10-65793192000186-175108-1
5,273,420A,3,273-420A-3-2024-04-10-65793192000186-175108-2
6,273,420A,4,273-420A-4-2024-04-19-54029423000163-12-0
7,273,420A,4,273-420A-4-2024-04-19-54029423000163-12-1
8,273,420A,4,273-420A-4-2024-04-19-54029423000163-12-2
9,273,420A,5,273-420A-5-2024-04-19-36664855000164-2189-0
